In [9]:
import os

In [10]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback

import os
import json
import pandas as pd
from dotenv import load_dotenv
import PyPDF2
import traceback

In [11]:
load_dotenv()

True

In [12]:
KEY = os.getenv("OPENAI_API_KEY")
KEY

'ASFJHSFKFDJHUFJFDDFLKJDFVLFJDFL'

In [ ]:
llm = ChatOpenAI(openai_api_key = key, model = 'gpt-3.5-turbo', temperature=0.7)
llm

In [19]:
# i need to read the response.json file , reading the json file is simple through file handling

with open("D:/project_1/response.json","r") as f:
    response_json = json.load(f)

In [20]:
print(response_json)

{'1': {'no': '1', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '2': {'no': '2', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '3': {'no': '3', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}}


In [ ]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [ ]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number", "subject","tone","response_json"],
    template = TEMPLATE
)

In [ ]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz",verbose = True)

In [ ]:
# again making the model to check whether the quiz questions created is right or not instead of making human to check

TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [ ]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject","quiz"],
    template = TEMPLATE2
)

In [ ]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review",verbose = True)

In [ ]:
generate_evaluation_chain = SequentialChain(chains=
    [quiz_chain, review_chain], 
    input_variables=["text","number", "subject","tone","response_json"],
    output_variables=["quiz","review"],
    verbose= True 
            )

In [14]:
# i have ramdomly taken some text from internet and stored that text in data.txt file
# i need to read the data.txt file

with open("D:\project_1\data.txt","r") as f:
    TEXT = f.read()

print(TEXT)


Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to:

Systems, architecture and infrastructure for Generative AI
Machine learning and Modeling using LLMs and Diffusion models
Large Language Models and its applications
Multi-modal Generative AI and its applications
Gen AI based Plugins and

In [21]:
# i need to generate MCQs on this text

TEXT
NUMBER=5
SUBJECT= "AI"
TONE = "Simple"
RESPONSE_JSON = response_json

In [24]:
# converting the RESPONSE_JSON in dictionary foramt to proper json foramt 

import json
json.dumps(RESPONSE_JSON)

'{"1": {"no": "1", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"no": "2", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"no": "3", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [ ]:

#  "https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking"

# track the total number of input tokens and output tokens along with the price


with get_openai_callback() as cb:
    response = generate_evaluation_chain(
        {
        "text": TEXT,
        "number": NUMBER,
        "subject" : SUBJECT,
        "tone": TONE,
        "response_json": RESPONSE_JSON
        }
    )

In [ ]:
print(response)

In [ ]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")


# Total Tokens:2452
# Prompt Tokens:1808
# Completion Tokens:644
# Total Cost:0.004

In [ ]:
quiz = response.get("quiz")
quiz

# this quiz is in string and i need to convert it into dictionary

In [ ]:
# convert quiz into dictionary

json.loads(quiz)

In [ ]:
# putting in proper format

quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
quiz_table_data

In [ ]:
quiz=pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("machinelearning.csv",index=False)

In [ ]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')